# 1 轨迹追踪

规划好路径以后（通常称为全局路径），全局路径由一系列路径点构成，这些路径点只要包含空间位置信息即可，也可以包含姿态信息，但是不需要与时间相关，这些路径点被称为全局路径点（Global Waypoint）。**路径（Path）**和**轨迹（Trajectory）**的区别就在于，轨迹还包含了时间信息，轨迹点也是一种路径点，它在路径点的基础上加入了时间约束，通产我们将这些轨迹点称为局部路径点（Local Waypoints）。

### 1.1 轨迹追踪问题的一般数学模型描述
- Switched系统（如足式机器人）的轨迹控制问题描述

$$
\begin{cases}
\min_{\textbf{u}(\cdot)} J = \sum_{i}\phi_i(\textbf{x}(t_{i+1})) + \int^{t_{i+1}}_{t_i}l_i(\textbf{x}(t),\textbf{u}(t),t)dt &\\
\text{s.t.} 
\quad \textbf{x}(t_0) = \textbf{x}_0 \quad &\text{initial state} \\
\quad \dot{\textbf{x}}(t) = f_i(\textbf{x}(t),\textbf{u}(t), t) \quad &\text{system flow map} \\
\quad \textbf{x}(t^{+}_{i+1}) = j(\textbf{x}(t_{i+1})) \quad &\text{system jump map}\\
\quad g_{1i}(\textbf{x}(t),\textbf{u}(t), t) = 0 \quad &\text{state-input equality constraints}\\
\quad g_{2i}(\textbf{x}(t), t) = 0 \quad &\text{state-only equality constraints}\\
\quad h_i(\textbf{x}(t),\textbf{u}(t), t)\geq 0 \quad &\text{inequality constraints}\\
\quad \text{where}\  t\in[t_i,t_{i+1}],\  i\in\{0,1,\cdots,I-1\}
\end{cases} 
$$
其中$t_i$是switching times;$t_I$是final time。

- 一般系统（如无人机、自动驾驶）的轨迹控制问题描述
$$
\begin{cases}
\min_{\textbf{u}(\cdot)} J = \phi(\textbf{x}(t_{I})) + \int^{t_{I}}_{t_0}l(\textbf{x}(t),\textbf{u}(t),t)dt &\\
\text{s.t.} 
\quad \textbf{x}(t_0) = \textbf{x}_0 \quad &\text{initial state} \\
\quad \dot{\textbf{x}}(t) = f(\textbf{x}(t),\textbf{u}(t), t) \quad &\text{system flow map} \\
\quad g_{1}(\textbf{x}(t),\textbf{u}(t), t) = 0 \quad &\text{state-input equality constraints}\\
\quad g_{2}(\textbf{x}(t), t) = 0 \quad &\text{state-only equality constraints}\\
\quad h(\textbf{x}(t),\textbf{u}(t), t)\geq 0 \quad &\text{inequality constraints}
\end{cases} 
$$

### 1.2 示范例子：无人车驾驶运动学模型（简化后轴原点模型）
动力学模型为
$$
\begin{cases}
\dot{x} &=  v\cos(\psi) \\
\dot{y} &= v\sin(\psi) \\
\dot{\psi} &= \frac{v}{L}\tan(\delta) \\
\dot{v}&= a
\end{cases}
$$
其中
- v为无人车的速度；
- $\dot{x}$ 为无人车在世界坐标系中X轴方向上的分速度，记为$v_{x}$ 
- $\dot{y}$ 为无人车在世界坐标系中Y轴方向上的分速度，记为$v_{y}$
- $\psi$ 为无人车在世界坐标中的航向角
- $\dot{\psi}$ 为无人车的角速度，可记为$\omega$

为了突显两个主要控制对象（速度v与角速度$\omega$），对以上无人车运动学模型进行变形，得以下形式：
$$
\begin{equation}
\left[ \begin{array}{cc}
        \dot{x} \\
        \dot{y} \\
        \dot{\psi}
        \end{array}
\right] = 
\left[ \begin{array}{cc}
        \cos(\psi) \\
        \sin(\psi) \\
        0
        \end{array}
\right] v +
\left[ \begin{array}{cc}
        0 \\
        0 \\
        1
        \end{array}
\right] \omega
\end{equation}
$$

#### 模型线性化：
$$
\begin{equation}
\color{blue}{(*): }
\dot{X} =
\left[ \begin{array}{cc}
        \dot{x} \\
        \dot{y} \\
        \dot{\psi} \\
        \dot{v}
        \end{array}
\right] = 
\left[ \begin{array}{cc}
        v\cos(\psi) \\
        v\sin(\psi) \\
        \frac{v\tan{\delta}}{L} \\
        a
        \end{array}
\right] 
= f(X,u)
\end{equation}
$$
其中$X=[x,y,\psi,v]^T,\ u=[a, \delta]^T$.

将$(*)$在$X_r=[x_r,y_r,\psi_r,v_r]^T, u_r=[a_r, \delta_r]^T$处展开并忽略二阶和高阶项，可得：
$$
\begin{align}
\dot{X} = f(X,u) &\approx f(X_r, u_r) + \frac{\partial f}{\partial X_r}(X-X_r) 
+ \frac{\partial f}{\partial u_r}(u-u_r) \\
& = \dot{X}_r + A_t(X-X_r) + B_t(u-u_r)
\end{align}
$$
其中
$$
A_t =
\left[
\begin{array}{cccc}
0 & 0 & -v_r\sin{\psi_r}& \cos{\psi_r}\\
0 & 0 & v_r\cos{\psi_r}& \sin{\psi_r}\\
0 & 0 & 0 & \frac{\tan{\delta_r}}{L}\\
0 & 0 & 0 & 0
\end{array}
\right],
\quad
B_t =
\left[
\begin{array}{cc}
0 & 0  \\
0 & 0  \\
0 & \frac{v_r}{L\cos^2{\delta_r}} \\
1 & 0
\end{array}
\right]
$$

记<font color='red'>$\Delta X=X-X_r,\ \Delta u =u-u_r$</font>, 则有
$$
\color{red}{(*):\quad \Delta \dot{X} = A_t\Delta X + B_t \Delta u}
$$

#### 线性模型离散化：
我们对$(*)$进行离散化，假设时间步长为$dt$，则有
$$
\begin{align}
\Delta X_{t+1} &= (I+dtA_t)\Delta X_t + dtB_t\Delta u_t \\
 &= \tilde{A}_t \Delta X_t + \tilde{B}_t \Delta u_t.
\end{align}
$$
其中
$$
\tilde{A}_t = I+dtA_t 
=\left[
\begin{array}{cccc}
1 & 0 & -v_r\sin{\psi_r}dt & \cos{\psi_r}dt\\
0 & 1 & v_r\cos{\psi_r}dt& \sin{\psi_r}dt\\
0 & 0 & 1 & \frac{\tan{\delta_r}}{L}dt\\
0 & 0 & 0 & 1
\end{array}
\right]
,
\quad \tilde{B}_t = dtB_t 
=\left[
\begin{array}{cc}
0 & 0  \\
0 & 0  \\
0 & \frac{v_r}{L\cos^2{\delta_r}}dt \\
dt & 0
\end{array}
\right]
.
$$

### *参考资料
> - [Welcome to PythonRobotics’s documentation!](https://atsushisakai.github.io/PythonRobotics/index.html)
> - [PythonRobotics](https://github.com/AtsushiSakai/PythonRobotics)
> - [无人驾驶之车辆控制（1）纯跟踪（Pure Pursuit）算法与Stanley算法](https://blog.csdn.net/zxxxxxxy/article/details/103665245)
> - [无人车系统（一）：运动学模型及其线性化](https://windses.blog.csdn.net/article/details/103463683)

# 2 纯追踪法（pure pursuit）
从自行车模型出发，纯跟踪算法以车后轴为切点, 车辆纵向车身为切线, 通过控制前轮转角 ， 使车辆可以沿着一条经过目标路点（goal point）的圆弧行驶，如下图所示：
![pure_pursuit](./figure/fig_pure_pursuit.png)

根据上图，我们有以下几何关系：
$$
\sin(\alpha) = \frac{l_d/2}{R}\ \Longrightarrow \ R=\frac{l_d}{2\sin(\alpha)}
$$
因为前轮转角$\delta$和轴距(wheel base)$L$之间有如下关系：
$$
\delta = \arctan(\frac{L}{R})
$$
从而我们可以得到
$$
\delta = \arctan(\frac{2L\sin\alpha}{l_d})
$$
其中，$\alpha$为路点与车后轴连成的向量的角度与车航向角的差值;d为车后轴离路点的距离，又被称为**前视距离**。

我们定义一个新的量$\textbf{e}_l$：车辆当前姿态和目标路点在横向上的误差。则从图中不难看出
$$
\sin(\alpha) = \frac{e_l}{l_d}
$$
从而得到
$$
\delta = \arctan(\frac{2Le_l}{l^2_d})
$$
通常来说，$l_d$被认为是车速的函数，在不同的车速下需要选择不同的前视距离。一种最常见的调整前视距离的方法就是将前视距离表示成车辆纵向速度的线形函数，即$l_d=kv_x$，那么前轮的转角公式就变成了：
$$
\delta = \arctan(\frac{2Le_l}{(kv_x)^2})
$$
那么纯追踪控制器的调整就变成了调整系数k，通常来说，会使用最大、最小前视距离来约束前视距离，越大的前视距离意味着轨迹的追踪越平滑，小的前视距离会使得追踪更加精确（当然也会带来控制的震荡）。

# 3 后轮反馈控制
<font color='red'>等待更新</font>

# 4 前轮反馈控制（Stanley method）
前面介绍的Pure Pursuit方法利用无人车的横向跟踪误差来设计控制器。在无人车轨迹跟踪任务中，除了横向跟踪误差外，是否还有其他信息有助于提高控制器的稳定性与有效性? 本篇介绍的Stanely方法就结合横向跟踪误差$e$与航向角偏差$\theta_e$来设计控制器。Stanley计算横向跟踪误差与pure pursuit方法是有区别的，Pure pursuit是以后轴中心为基准点计算几何学公式，而Stanley是基于前轴中心为基准点计算几何学公式的。Stanley方法基于横向跟踪误差（cross-track error: $e$）的非线性反馈函数，并且能实现横向跟踪误差指数收敛于0. 如图所示
![stanley](./figure/stanley.jpeg)

根据车辆位姿与给定路径的相对几何关系可以直观的获得控制车辆方向盘转角的控制变量，其中包含横向偏差$e$和航向偏差$\theta_e$
$$
\delta = \delta_e + \delta_{\theta_e}
$$
其中
  - 不考虑横向跟踪误差的情况下，前轮偏角和给定路径切线方向一致；$\theta_e$表示车辆航向与最近路径点切线方向之间的夹角，在没有任何横向误差的情况下，前轮方向与所在路径点的方向相同：$$\delta_{\theta_e} = \theta_e$$
  - 不考虑航向跟踪偏差的情况下，横向跟踪误差越大，前轮转向角越大;假设车辆预期轨迹在距离前轮$d(t)$处与给定路径上最近点切线相交，根据几何关系得出如下非线性比例函数：
    $$
    \delta_e = \arctan\frac{e(t)}{d(t)} = \arctan\frac{ke(t)}{v(t)}
    $$
    其中$d(t)$与车速相关，最后用车速$v(t)$、增益参数$k$表示.
综合可得
$$
\delta = \arctan\frac{ke(t)}{v(t)} + \theta_e
$$

### 3.1 stanley 方法的收敛性
使用线性自行车运动模型，可以得到横向误差的变化率：
$$
\dot{e}(t) = -v(t)\sin\delta_e(t)
$$
由几何关系可知：
$$
\sin\delta_e(t) = \frac{e(t)}{\sqrt{d^2(t)+e^2(t)}} = \frac{ke(t)}{\sqrt{v^2(t)+k^2e^2(t)}}
$$
因此有
$$
\dot{e}(t)= \frac{-ke(t)v(t)}{\sqrt{v^2(t)+k^2e^2(t)}} = \frac{-ke(t)}{\sqrt{1+(\frac{ke(t)}{v(t)})^2}}
$$
当横向跟踪误差$e(t)$很小时，$(ke(t)/v(t))^2\rightarrow 0$:
$$
\dot{e}(t)\approx -ke(t) \Longrightarrow e(t)=e_0\exp^{-kt}
$$
因此横向误差指数收敛于$e(t)=0$，参数$t$决定了收敛速度。